# Assigment 2

## Analysing Data base 

In [1]:
import pandas as pd
import sqlite3

In [2]:
try:
    connection=sqlite3.connect('lahman2014.sqlite')
except Error as e:
    print(e)

In [3]:
# We found that we have only salary from period 1985 to 2014\
#SO,first we extracted data of teams between this period to analyze it
teamPandas=pd.read_sql_query("""SELECT yearID,teamID,Rank,G,W,name FROM Teams where yearID BETWEEN 1985 and 2014""",connection)
teamPandas.head()

,yearID,teamID,Rank,G,W,name
0,1985,BAL,4,161,83,Baltimore Orioles
1,1985,BOS,5,163,81,Boston Red Sox
2,1985,CAL,2,162,90,California Angels
3,1985,CHA,3,163,85,Chicago White Sox
4,1985,CLE,7,162,60,Cleveland Indians


In [4]:
#Reading data of salaries
salaryPandas=pd.read_sql_query("""SELECT * FROM Salaries""",connection)
salaryPandas.head()

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000.0
1,1985,ATL,NL,bedrost01,550000.0
2,1985,ATL,NL,benedbr01,545000.0
3,1985,ATL,NL,campri01,633333.0
4,1985,ATL,NL,ceronri01,625000.0


In [5]:
dictionaryOfTeams={}
for i in teamPandas.itertuples():
    if i[2] not in dictionaryOfTeams:
        #2 for teamId
        #4 for gamePlay
        #5 for wining
        dictionaryOfTeams[i[2]]=[i[4],i[5],0]
    else:
        dictionaryOfTeams[i[2]][0]+=i[4]
        dictionaryOfTeams[i[2]][1]+=i[5]
#For loop to add all salaries to leagueTeams
for i in salaryPandas.itertuples():
    if i[2] not in dictionaryOfTeams:
        continue
    dictionaryOfTeams[i[2]][2]+=i[5]

In [6]:
len(dictionaryOfTeams)

35

In [7]:
#Just merging data by pandas to ensure that data from sql is right
TeamPayRoll1=pd.DataFrame.from_dict(dictionaryOfTeams, orient='index',columns=['Playing', 'Winning','Total payRolls in million'])

In [8]:
TeamPayRoll1['Total payRolls in million'] = TeamPayRoll1['Total payRolls in million'].astype(float)
TeamPayRoll1['Total payRolls in million']/=1000000

In [9]:
TeamPayRoll1['WinRate']=TeamPayRoll1.Winning/TeamPayRoll1.Playing

In [10]:
TeamPayRoll1.head()

,Playing,Winning,Total payRolls in million,WinRate
BAL,4790,2265,1630.200038,0.472860
BOS,4794,2576,2432.700935,0.537338
CAL,1879,922,271.978930,0.490687
CHA,4794,2435,1699.865143,0.507927
CLE,4792,2389,1405.016128,0.498539


In [11]:
#those teams have different names in teamDataBase and salary data base 
notFoundTeams=[]
for i in salaryPandas.itertuples():
     if i[2] not in dictionaryOfTeams and i[2]not in notFoundTeams:
            notFoundTeams.append(i[2])
notFoundTeams

['NYM', 'SFG']

## Part 1

In [12]:
teamPayRoll2=pd.read_sql_query(""" 
SELECT Teams.teamID,Teams.name, CAST(sum(Teams.W) AS FLOAT)/CAST(sum(Teams.G) AS FLOAT) AS 'WinningRatio', sum(salary)/1000000 AS 'Total Pay in Million'
FROM Teams Left join Salaries
ON Teams.teamID = Salaries.teamID And Teams.yearID = Salaries.yearID
GROUP BY Teams.teamID
""",connection)

In [13]:
teamPayRoll2.head()

,teamID,name,WinningRatio,Total Pay in Million
0,ALT,Altoona Mountain City,0.240000,NaN
1,ANA,Anaheim Angels,0.508297,468.091973
2,ARI,Arizona Diamondbacks,0.493687,1209.975228
3,ATL,Atlanta Braves,0.540464,1882.946784
4,BAL,Baltimore Orioles,0.476154,1630.200038


In [14]:
teamPayRoll2.loc[teamPayRoll2.teamID=='SFN']

,teamID,name,WinningRatio,Total Pay in Million
116,SFN,San Francisco Giants,0.519432,1696.242921


In [15]:
teamPayRoll2.loc[teamPayRoll2.teamID=='SFN']


,teamID,name,WinningRatio,Total Pay in Million
116,SFN,San Francisco Giants,0.519432,1696.242921


In [16]:
teamPayRoll2.loc[teamPayRoll2.teamID=='NYN']

,teamID,name,WinningRatio,Total Pay in Million
93,NYN,New York Mets,0.508664,1965.737231


In [17]:
teamPayRoll2=teamPayRoll2.drop(teamPayRoll2.index[93])
teamPayRoll2=teamPayRoll2.drop(teamPayRoll2.index[116])

In [18]:
newSfn=pd.read_sql_query(""" 
SELECT Teams.teamID,name,Cast(sum(w) AS FLOAT)/sum(g) as'WinningRatio',sum(salary)/1000000 AS 'Total Pay in Million'
FROM Teams,Salaries
where (teamidbr='SFG' or teamidbr='NYM' )and teams.yearID=Salaries.yearID AND (teams.teamID=Salaries.teamID or Salaries.teamid=teams.teamidbr)and Teams.yearid between 1985 and 2014
group by teamidbr
""",connection)

In [19]:
newSfn

,teamID,name,WinningRatio,Total Pay in Million
0,NYN,New York Mets,0.510081,2020.544221
1,SFN,San Francisco Giants,0.520245,1839.753088


In [20]:
lastPayRoll=pd.concat([teamPayRoll2, newSfn])

In [21]:
lastPayRoll.to_csv('lastPayRoll.csv')

In [22]:
lastPayRoll.head()

,teamID,name,WinningRatio,Total Pay in Million
0,ALT,Altoona Mountain City,0.240000,NaN
1,ANA,Anaheim Angels,0.508297,468.091973
2,ARI,Arizona Diamondbacks,0.493687,1209.975228
3,ATL,Atlanta Braves,0.540464,1882.946784
4,BAL,Baltimore Orioles,0.476154,1630.200038


In [25]:
df = lastPayRoll.dropna();df.head()

,teamID,name,WinningRatio,Total Pay in Million
1,ANA,Anaheim Angels,0.508297,468.091973
2,ARI,Arizona Diamondbacks,0.493687,1209.975228
3,ATL,Atlanta Braves,0.540464,1882.946784
4,BAL,Baltimore Orioles,0.476154,1630.200038
15,BOS,Boston Americans,0.534566,2432.700935


In [29]:
import plotly.express as px
fig = px.scatter(df, y="WinningRatio", x="Total Pay in Million", color='name')
fig.show()